In [1]:
import numpy as np

from mpl_toolkits.mplot3d import Axes3D

import matplotlib as mpl
import matplotlib.pyplot as plt
import App.OctLayers as oct
import os
import pdb

%matplotlib inline
mpl.rcParams['figure.figsize'] = (10.0, 8.0)

In [2]:
filepath = 'Data/Sample/Cirrus/Sample1/Macular Cube 512x128_01-01-2001_01-01-01_OS_sn41343_cube_z'
img_file = filepath + '.img'
surfaces_file = os.path.join(filepath,'Macular Cube 512x128_01-01-2001_01-01-01_OS_sn41343_cube_z_Surfaces_Iowa.xml')
center_file = os.path.join(filepath,'Macular Cube 512x128_01-01-2001_01-01-01_OS_sn41343_cube_z_GridCenter_Iowa.xml')

data_oct = oct.OctLayers(filename = surfaces_file,
                         center_filename = center_file,
                         raw_filename = img_file)

In [ ]:
def find_thickness(surface2, start_pixel, vector, step=0.1):
    #assert vector[2] > 0, "Vector in the wrong direction"
    found = False
    t = 0
    
    while not found:
        t = t + step
        scaled_vec = t * vector
        new_pix = scaled_vec + start_pixel
        
        low_x = np.floor(new_pix[0])
        high_x = np.ceil(new_pix[0])
        low_y = np.floor(new_pix[1])
        high_y = np.ceil(new_pix[1])
        #import pdb; pdb.set_trace()
        
        # First check the new x,y coordinates are valid. If not return np.nan
        if new_pix[0] > surface2.shape[1] - 1  or new_pix[1] > surface2.shape[0] - 1 \
           or new_pix[0] < 0 or new_pix[1] < 0:
            return np.nan
        
        # Check if both vector components are 0
        if np.isclose(scaled_vec[0], 0) and np.isclose(scaled_vec[1], 0):
            # no interpolation required
            new_val = surface2[new_pix[1], new_pix[0]]
            
        # now check if the new pixels fall exactly on a real pixel (ie. new_pix is an integer)
        if not (np.isclose(new_pix[0], int(new_pix[0])) or np.isclose(new_pix[1], int(new_pix[1]))):
            # bilinear interpolation
            points = [(low_x,low_y,surface2[low_y,low_x]),
                      (low_x,high_y,surface2[low_y,high_x]),
                      (high_x,low_y,surface2[high_y,low_x]),
                      (high_x,high_y,surface2[high_y,high_x])]
            new_val = bilinear_interpolation(new_pix[0], new_pix[1], points)
            
        elif np.isclose(new_pix[0], int(new_pix[0])):
            # only x pixel exists, linear interp on y
            new_val = np.interp(new_pix[1], [low_y,high_y],[surface2[low_y,new_pix[0]],
                                                            surface2[high_y,new_pix[0]]])
        elif np.isclose(new_pix[1], int(new_pix[1])):
            # only y vector component is 0, linear interp on x
            new_val = np.interp(new_pix[0], [low_x,high_x],[surface2[new_pix[1],low_x],
                                                            surface2[new_pix[1],high_x]])
        else:
            # no interpolation required
            new_val = surface2[start_pixel[1], start_pixel[0]]
                        
            
        if new_pix[2] >= new_val:
            vec_length = np.linalg.norm(t * vector)
            return t
        
def bilinear_interpolation(x, y, points):
    '''Interpolate (x,y) from values associated with four points.

    The four points are a list of four triplets:  (x, y, value).
    The four points can be in any order.  They should form a rectangle.

        >>> bilinear_interpolation(12, 5.5,
        ...                        [(10, 4, 100),
        ...                         (20, 4, 200),
        ...                         (10, 6, 150),
        ...                         (20, 6, 300)])
        165.0

    '''
    # See formula at:  http://en.wikipedia.org/wiki/Bilinear_interpolation

    points = sorted(points)               # order points by x, then by y
    (x1, y1, q11), (_x1, y2, q12), (x2, _y1, q21), (_x2, _y2, q22) = points

    if x1 != _x1 or x2 != _x2 or y1 != _y1 or y2 != _y2:
        pdb.pm()
        raise ValueError('points do not form a rectangle')
        
    if not x1 <= x <= x2 or not y1 <= y <= y2:
        raise ValueError('(x, y) not within the rectangle')

    return (q11 * (x2 - x) * (y2 - y) +
            q21 * (x - x1) * (y2 - y) +
            q12 * (x2 - x) * (y - y1) +
            q22 * (x - x1) * (y - y1)
           ) / ((x2 - x1) * (y2 - y1) + 0.0)



In [ ]:
data_oct.data.data.shape

In [ ]:
my_surface = data_oct.data.data[0,:,:]
my_surface2 = data_oct.data.data[10,:,:]

In [ ]:
# sample surface
#my_surface = np.array(np.arange(30), dtype=np.float).reshape(5,6)
#my_surface = np.random.random((5,6))
#my_surface = np.random.random((1024,1000))
#my_surface2 = my_surface + np.random.random(my_surface.shape)*10

calc_size = 10


# start and end points for each vector across a point
points = np.zeros((4,2),
                  dtype=[('x', 'i4'),('y', 'i4'),('z','f8')])
points[:,0]['x'] = [-10, 0, 10, 10]
points[:,0]['y'] = [10, 10, 10, 0]
points[:,1]['x'] = [10, 0, -10, -10]
points[:,1]['y'] = [-10, -10, -10, 0] 



# points[:,0][0:2] = [(-1,1),(0,1),(1,1),(1,0)]
# points[:,1][0:2] = [(1,-1),(0,-1),(-1,-1),(-1,0)]

# not going to calulate vectors for pixels on the edges of the surface
valid_size = (my_surface.shape[0]-(calc_size * 2)) * (my_surface.shape[1]-(calc_size * 2))

# the points array for each valid pixel
points = np.repeat(np.expand_dims(points,2),valid_size,2)

coords_x = range(calc_size,my_surface.shape[1]-calc_size)
coords_y = range(calc_size,my_surface.shape[0]-calc_size)

xv, yv = np.meshgrid(coords_x,coords_y, indexing='ij')

points[:,0,:]['x'] = points[:,0,:]['x'] + xv.flatten()
points[:,1,:]['x'] = points[:,1,:]['x'] + xv.flatten()
points[:,0,:]['y'] = points[:,0,:]['y'] + yv.flatten()
points[:,1,:]['y'] = points[:,1,:]['y'] + yv.flatten()


# points is now a 3d array [a, b, c]
# each datapoint is a coordinate (x,y,z)
# a = 4 - The number of vectors crossing each pixel
# b = 2 - Point coordinates of the end of each vector (0=start, 1=end)
# c = N number of valid pixels in the surface

# extract the z coordinates
x = points['x']
y = points['y']
points['z'] = my_surface[y,x]
#import pdb; pdb.set_trace()
# Calculate the vectors
vectors = np.empty((4,points.shape[2]),
                   dtype=[('x','f8'),('y','f8'),('z','f8')])
vectors['x'] = points[:,1,:]['x'] - points[:,0,:]['x']
vectors['y'] = points[:,1,:]['y'] - points[:,0,:]['y']
vectors['z'] = points[:,1,:]['z'] - points[:,0,:]['z']

# Generate the vector pairs
pairs = np.array([(0,1),(0,2),(0,3),(1,2),(1,3),(2,3)], dtype=np.int)

normal_vector = np.empty(valid_size, dtype=(np.float,3))
vector_pairs = np.empty((6,2), dtype=(np.float,3))
normal_vectors = np.empty(my_surface.shape, dtype=(np.float,3))
normal_vectors[:] = np.nan
thicknesses = np.empty(my_surface.shape)
thicknesses[:] = np.nan

#voxel_sizes = [v for i,v in data_oct.voxel_size.items()]
voxel_sizes = [data_oct.voxel_size['x'], data_oct.voxel_size['y'], data_oct.voxel_size['z']]

In [ ]:
old_pcomp = 0.0
for i in range(valid_size):
    pcomp = (float(i) / valid_size)*100
    if int(pcomp) > old_pcomp:
        old_pcomp = pcomp
        print('percent: {}'.format(int(pcomp)))
    # need to convert from structured arrays (np.void) to ndarray
    vector_pairs[:, 0, :] = np.matrix([vectors[pairs[:,0], i]['x'],
                                       vectors[pairs[:,0], i]['y'],
                                       vectors[pairs[:,0], i]['z']]).transpose()
    vector_pairs[:, 1, :] = np.matrix([vectors[pairs[:,1], i]['x'],
                                       vectors[pairs[:,1], i]['y'],
                                       vectors[pairs[:,1], i]['z']]).transpose()
    
    # convert the vector pairs from coordinates to microns
    #import pdb; pdb.set_trace()
    #vector_pairs[:, 0, :] = vector_pairs[:, 0, :] * voxel_sizes
    #vector_pairs[:, 1, :] = vector_pairs[:, 1, :] * voxel_sizes
    
    # calculate a normal vector for each vector pair
    potential_vectors = np.cross(vector_pairs[:, 0],
                              vector_pairs[:, 1])
    
    #import pdb; pdb.set_trace()
    # average the 6 normal vectors to find the mean value for the pixel
    normal_vector = potential_vectors.mean(axis=0) 
    
    # Normalise the vector to unit length
    normal_vector = normal_vector / np.linalg.norm(normal_vector)
    
    # Convert the vectors back into pixel coordinates
    #normal_vector = normal_vector / voxel_sizes
    
    # The normal vector could be in either direction
    # TODO: need to add a check for direction
    if normal_vector[2] < 0:
        normal_vector = 0 - normal_vector
        
    # create an array to hold the output thickness values
    #thicknesses = np.empty((my_surface.shape[0]-2,my_surface.shape[1]-2),
                           #dtype=(np.float,3))

    # note output from unravel is [y,x]
    start_pixel = np.unravel_index(i, (my_surface.shape[0]-(2 * calc_size),my_surface.shape[1]-(2 * calc_size)))
    
    # append the origin z coordinate to the start pixel
    surface_pixel = [val + calc_size for val in start_pixel]
    surface_pixel = surface_pixel[::-1]
    surface_pixel.append(my_surface[surface_pixel[1], surface_pixel[0]])
    
    # Store the normal vector incase I want it later
    normal_vectors[surface_pixel[1], surface_pixel[0]] = normal_vector
    #import pdb;pdb.set_trace()
    thicknesses[surface_pixel[1], surface_pixel[0]] = find_thickness(my_surface2, surface_pixel, normal_vector)

In [ ]:
normal_vectors[18:27,25]


In [ ]:
fig = plt.figure()
ax = fig.gca(projection='3d')
X, Y = np.meshgrid(np.arange(my_surface.shape[1]),
                   np.arange(my_surface.shape[0]))
Z = my_surface

surf = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, cmap = mpl.cm.coolwarm,
                       linewidth=0, antialiased=False)

ax.set_zlim(my_surface.min(), my_surface2.max())

for i in range(my_surface.size):
    y,x = np.unravel_index(i, my_surface.shape)
    X=[x,x]
    Y=[y,y]
    Z = [my_surface[y,x], my_surface2[y,x]]
    ax.plot(X,Y,Z, '--k')

for i in range(len(normal_vectors)):
    y,x = np.unravel_index(i, (my_surface.shape[0]-(2 * calc_size), my_surface.shape[1]-(2* calc_size)))
    y = y + calc_size
    x = x + calc_size
    v = normal_vectors[y,x]
    t = thicknesses[y, x]
    v = (v * t) + [x,y,my_surface[y,x]]
    X = [x, v[0]]
    Y = [y, v[1]]
    Z = [my_surface[y,x], v[2]]
    ax.plot(X,Y,Z, 'r')

X, Y = np.meshgrid(np.arange(my_surface.shape[1]),
               np.arange(my_surface.shape[0]))
Z = my_surface2
surf2 = ax.plot_surface(X, Y, Z, rstride=1, cstride=1, antialiased=False)
    
plt.show()

In [ ]:
# extract one slice of the raw oct and convert to RGB
my_oct = data_oct.octdata[64,:,:]
my_oct = np.expand_dims(my_oct,2)
my_oct = np.tile(my_oct,(1,1,3))

#my_oct = np.flipud(my_oct)

In [ ]:
my_oct.shape

In [ ]:
thicknesses.shape

In [ ]:
vector_slice = normal_vectors[14,:]
for iline in range(0,100,10):
    X = iline + 150
    Y = vector_slice[iline,2] * thicknesses[14, iline]

In [ ]:
vector_slice[10,2] * thicknesses[14,10]

In [ ]:
fig = plt.figure()
ax = fig.gca()
ax.imshow(my_oct)
ax.autoscale(False)
ax.plot(range(150,250),my_surface[14,:],'--r')
ax.plot(range(150,250),my_surface2[14,:],'--g')

In [ ]:
fig = plt.figure()
ax = fig.gca()
ax.imshow(my_oct)
ax.autoscale(False)
ax.plot(range(150,250),my_surface[14,:],'--r')
ax.plot(range(150,250),my_surface2[14,:],'--r')

for iline in range(0,100,10):
    X_start = iline + 150
    X_end = X_start + vector_slice[iline,0] * thicknesses[14, iline]
    Y_start = my_surface[14, iline]
    Y_end = Y_start + vector_slice[iline,2] * thicknesses[14, iline]
    ax.plot([X_start, X_end],[Y_start,Y_end],'b')

In [ ]:
iline = 20

In [ ]:
X_start = iline + 150
X_end = X_start + vector_slice[iline,0] * thicknesses[14, iline]
[X_start, X_end]

In [ ]:
vector_slice[iline,:]

In [ ]:
thicknesses[14, iline]

In [ ]:
X_end

In [ ]:
X_start

In [ ]:
np.all(np.isnan(thicknesses))

In [ ]:
thicknesses